## Prerequisites

In [ ]:
!pip install numpy matplotlib pandas sklearn

In [ ]:
!python -V

## Data Preprocessing

In [ ]:
import scipy
from scipy.io import loadmat
import pickle

### Input dataset

In [ ]:
data = loadmat(r'Au-Ti-SiC-dataset.mat')

In [ ]:
import pickle
import numpy as np

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
X = np.transpose(data['re'])
X.shape

In [ ]:
X = X.T
X.shape

In [ ]:
layer_1 = np.linspace(250, 500, 100, endpoint=True)
layer_4 = np.linspace(0.1, 1.5, 100, endpoint=True)
layer_2 = layer_4*100

In [ ]:
y = np.zeros((X.shape[0], 2))

In [ ]:
i = 0
for l2 in layer_2:
    for l1 in layer_1:
            y[i] = [ l2, l1]
            i = i + 1

In [ ]:
y.shape

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [ ]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [ ]:
x_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

x_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [ ]:
# Builds dataset with ALL data
origin_train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(origin_train_dataset, [int(x_train_tensor.shape[0] * 0.9), int(x_train_tensor.shape[0] * 0.1)])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=2000)
val_loader = DataLoader(dataset=val_dataset, batch_size=2000)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader  = DataLoader(dataset=test_dataset, batch_size=2000)

## Model

In [ ]:
import torch.nn as nn

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm1d(X.shape[1])
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        return x

## Training

In [ ]:
import torch.optim as optim

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
n_epochs = 250

In [ ]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yh = model(x)
        loss = loss_fn(y, yh)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [ ]:
model = Net().to(device)

loss_fn = nn.MSELoss(reduction='mean')

# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

train_step = make_train_step(model, loss_fn, optimizer)

In [ ]:
model.eval()

In [ ]:
training_losses = []
validation_losses = []

for epoch in range(n_epochs):
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yh = model(x_val)
            #yh = torch.reshape(yh, (-1,))
            val_loss = loss_fn(y_val, yh).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.5f}\t Validation loss: {validation_loss:.5f}")

## Testing

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
x_test_tensor = x_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
y_pred = model(x_test_tensor).squeeze()

In [ ]:
test_loss = loss_fn(y_test_tensor, y_pred)
print(test_loss)

In [ ]:
print(f"The mean of absolute percentage error: {mean_absolute_percentage_error(y_test_tensor.cpu(), y_pred.cpu()):.2f}%")

## Real-world case predict

In [ ]:
import pandas as pd

### Input experimental data

In [ ]:
case1 = pd.read_csv('Au-Ti-SiCexperimentaldata.csv', header=None)
case1 = np.reshape(np.array(case1), [ 1,-1])

In [ ]:
case1.shape

In [ ]:
case1= np.array(case1)
case1_tensor = torch.from_numpy(case1).float()

In [ ]:
model.eval()
pred_1 = model(case1_tensor)

### Output predction

In [ ]:
pred_1